# Final Project DL model Notebook - Retrain Model From User Data

improve and adapt model using data collected in the real world

In [14]:
import sys

from keras.models import load_model
import numpy as np

from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils import *

# Model and data dirs
current_dir = Path.cwd()

HOME_DIR = current_dir.parent
DATA_DIR = HOME_DIR.joinpath('data/')
VAL_DATA_PATH = DATA_DIR.joinpath('valid')
RETRAIN_DATA_DIR = DATA_DIR.joinpath('retrainBatch01-28-02-2018/labelled/')
MODEL_NAME = 'CNN_two_convs_30122017_1700.h5'
MODEL_DIR = HOME_DIR.joinpath('savedModels/')
MODEL = MODEL_DIR.joinpath(MODEL_NAME)

# training variables
batch_size = 64
epochs = 15
learning_rate = 1e-4

In [15]:
model = load_model(MODEL)
model.summary()

/Users/pgieschen/anaconda/envs/finalProject/lib/python3.6/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 224, 224)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 222, 222)      896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 222, 222)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 74, 74)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 72, 72)        18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 72, 72)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 24, 24)        0         
__________

## Validate current performance of the loaded model

In [23]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()
validation_generator = datagen.flow_from_directory(
        VAL_DATA_PATH,
        target_size=(224, 224),
        batch_size=batch_size)

Found 5000 images belonging to 10 classes.


In [ ]:
val_batches = get_in_batches(VAL_DATA_PATH, shuffle=False, class_mode=None, batch_size=batch_size)
pred_classes = model.evaluate_generator(validation_generator, 5000)
# pred_classes = np.argmax(pred_classes, axis=1)
# act_classes = val_batches.classes
#print(pred_classes)

In [18]:
retrain_batches = get_in_batches(RETRAIN_DATA_DIR, batch_size=batch_size, augment=True)
retrain_classes = retrain_batches.classes
retrain_labels = onehot(retrain_batches.classes)

Found 181 images belonging to 10 classes.


In [ ]:
history = model.fit_generator(retrain_batches,
                    steps_per_epoch=retrain_batches.n//retrain_batches.batch_size,
                    epochs=5,
                    callbacks=[reduce_lr, stop_early])